In [2]:
!pip install filelock fsspec[http] huggingface-hub pandas tqdm xxhash datasets

In [1]:
# Use !pip install if running directly in Jupyter
!pip install langchain langchain-groq langchain_community tavily-python python-dotenv langchainhub -q

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Optional: Check if keys are loaded (recommended)
groq_api_key = os.getenv("GROQ_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

if not groq_api_key:
    print("GROQ_API_KEY not found. Make sure it's set in your .env file.")
if not tavily_api_key:
    print("TAVILY_API_KEY not found. Make sure it's set in your .env file.")

# Note: Don't print the actual keys here or in logs for security!
print("API Keys loaded (presence checked).")

API Keys loaded (presence checked).


In [17]:
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize the LLM (using Groq's Llama3 8b instruct model - fast and capable)
# You can also try "mixtral-8x7b-32768"
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0) # Temperature 0 for more deterministic output

# Initialize the Tavily Search Tool
# max_results=3 asks Tavily to return the top 3 search results
search_tool = TavilySearchResults(max_results=3)

# Put the tool(s) into a list
tools = [search_tool]

# Let's test the tool independently (optional)
# print(search_tool.invoke("What are the latest advancements in battery technology?"))

In [18]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

# 1. Get the ReAct prompt template
# This prompt contains instructions for the LLM on how to use tools and formulate thoughts/actions
prompt = hub.pull("hwchase17/react") # A common, well-tested ReAct prompt

# 2. Create the Agent
# This function binds the LLM, tools, and prompt together into a runnable agent logic
agent = create_react_agent(llm, tools, prompt)

# 3. Create the Agent Executor
# This is the runtime for the agent. It takes the agent logic and executes the steps.
# verbose=True lets us see the agent's thought process (chain-of-thought)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Agent Executor created successfully.")

D:\ANACONDA\Lib\site-packages\langsmith\client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Agent Executor created successfully.


In [20]:
# Define the research topic
input_topic = "Whay Pakistan is important for this world?"

# Invoke the agent executor
# The agent will now:
# 1. See the input topic.
# 2. Decide it needs to search the web (using Tavily).
# 3. Formulate a search query.
# 4. Call the Tavily tool.
# 5. Get the search results.
# 6. Process the results using the LLM based on the original topic.
# 7. Generate the final answer (summary).

response = agent_executor.invoke({"input": input_topic})

print("\n--- Final Answer ---")
print(response['output'])



> Entering new AgentExecutor chain...
Thought: Pakistan is a country with a rich history and strategic location, but what makes it important for the world?

Action: tavily_search_results_json
Action Input: "Pakistan's importance in the world"
[{'title': 'The Strategic Importance of Pakistan: A Geostrategic Nexus', 'url': 'https://bestdiplomats.org/strategic-importance-of-pakistan/', 'content': 'In conclusion, Pakistan’s strategic importance is deeply rooted in its geographical location, economic potential, and security dynamics. Its role as a bridge between South Asia and Central Asia, its contribution to regional stability, and its partnerships with global powers like China and the United States highlight its significance on the world stage. [...] Best\n\n\n\t\t\t\tDiplomats\n\nThe Strategic Importance of Pakistan: A Geostrategic Nexus\n\nPakistan’s strategic importance lies in its geographical position at the intersection of South Asia, Central Asia, and the Middle East. It shares 